# Modeling

In [1]:
import pandas as pd
import numpy as np

In [2]:
import cufflinks as cf; cf.go_offline()

## Dataset

In [3]:
df = pd.read_feather('removed_duplicated_columns.feather')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726288 entries, 0 to 726287
Columns: 767 entries, HKTLMYY to TLJYWBE
dtypes: float64(742), int64(21), object(4)
memory usage: 4.2+ GB


In [4]:
target = 'TLJYWBE'
df[target] = df[target].apply(lambda v: v >= 1e-5)
df[target].value_counts()

False    726224
True         64
Name: TLJYWBE, dtype: int64

In [5]:
negatives = df[~df[target]]
negatives[target].value_counts()
dataset = negatives.sample(frac=0.2).append(df[df[target]])
dataset[target].value_counts()

False    145245
True         64
Name: TLJYWBE, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
features = [c for c in df.columns if c != target]
training_features, test_features, train_target, test_target = train_test_split(dataset[features], dataset[target], test_size=0.25, stratify=dataset[target])

In [8]:
print(f'Training set shape: {training_features.shape}')
print(f'Training set target shape: {train_target.shape}')
print(f'Test set shape: {test_features.shape}')
print(f'Training set target shape: {test_target.shape}')

Training set shape: (108981, 766)
Training set target shape: (108981,)
Test set shape: (36328, 766)
Training set target shape: (36328,)


In [9]:
test_target.value_counts()

False    36312
True        16
Name: TLJYWBE, dtype: int64

## Modeling

In [10]:
dtypes = df.dtypes
dtypes.head()

HKTLMYY      int64
IJEXXXL    float64
KVNLYTZ    float64
ADOIOYN    float64
GPCKISJ    float64
dtype: object

In [22]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [12]:
int_features = dtypes[dtypes == 'int64'].index
float_features = dtypes[dtypes == 'float64'].index
cat_features = dtypes[dtypes == 'object'].index
len(int_features), len(float_features), len(cat_features)

(21, 741, 4)

In [13]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('int', numeric_transformer, int_features),
        ('float', numeric_transformer, float_features),
        ('cat', categorical_transformer, cat_features)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', 'passthrough')  
                          ])

In [ ]:
%%time
param_grid = [
    {
        'classifier': [RidgeClassifier()],
        'classifier__alpha': [0.1, 1, 10, 100, 200, 1000],  
        'classifier__class_weight': [{0: 1, 1: w} for w in [10, 50, 100, 150, 200]]
    },
    {
        'classifier': [LogisticRegression()],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__C': [0.1, 1, 10, 100, 200, 1000],
        'classifier__solver': ['saga'],
        'classifier__class_weight': [{0: 1, 1: w} for w in [10, 50, 100, 150, 200]]
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__class_weight': ['balanced', 'balanced_subsample', None]
    }
]


search = RandomizedSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=2)
search.fit(training_features, train_target)

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
from sklearn.metrics import f1_score, precision_score
best_estimator = search.best_estimator_
best_estimator.fit(training_features, train_target)
test_predictions = best_estimator.predict(test_features)
f1 = f1_score(test_target, test_predictions)
precision = precision_score(test_target, test_predictions)
print(f"F1 Score on Test Set: {f1:.3f}")
print(f"Precision on Test Set: {precision:.3f}")

In [18]:
from sklearn.metrics import f1_score, precision_score
best_estimator = search.best_estimator_
best_estimator.fit(training_features, train_target)
test_predictions = best_estimator.predict(test_features)
f1 = f1_score(test_target, test_predictions)
precision = precision_score(test_target, test_predictions)
print(f"F1 Score on Test Set: {f1:.3f}")
print(f"Precision on Test Set: {precision:.3f}")

F1 Score on Test Set: 0.114
Precision on Test Set: 0.105


## Feature importance

In [46]:
feature_names = training_features.columns.tolist()
coefficients = best_estimator.named_steps['classifier'].coef_[0]
feature_importance = dict(zip(feature_names, coefficients))
sorted_features = sorted(feature_importance.items(), key=lambda x: abs(x[1]), reverse=True)

for i, (feature, coef) in enumerate(sorted_features):
    print(f"Feature {i}: {feature}, Coefficient: {coef:.4f}")

Feature 0: OKIKPOJ, Coefficient: -0.2588
Feature 1: AEXRRBM, Coefficient: 0.2236
Feature 2: SHEGNUT, Coefficient: 0.1744
Feature 3: FPGUGZC, Coefficient: -0.1654
Feature 4: FFJOGRA, Coefficient: 0.1638
Feature 5: ALVJYOA, Coefficient: 0.1572
Feature 6: FBDGIUT, Coefficient: 0.1496
Feature 7: IJTGMGJ, Coefficient: -0.1423
Feature 8: MUEKGHC, Coefficient: -0.1341
Feature 9: TXEIVQY, Coefficient: -0.1325
Feature 10: OULTOYT, Coefficient: -0.1296
Feature 11: LWFSOBB, Coefficient: -0.1288
Feature 12: GYXMTQR, Coefficient: -0.1274
Feature 13: DVSZBLN, Coefficient: 0.1250
Feature 14: CWWUCQG, Coefficient: 0.1184
Feature 15: FTFSHCI, Coefficient: -0.1149
Feature 16: QNIBUZZ, Coefficient: 0.1137
Feature 17: SRZFXSA, Coefficient: 0.1058
Feature 18: KMGNWGY, Coefficient: -0.1029
Feature 19: AHBBVON, Coefficient: -0.0987
Feature 20: TNLTITB, Coefficient: 0.0985
Feature 21: ZPBDTHO, Coefficient: 0.0971
Feature 22: RRMMRAJ, Coefficient: 0.0920
Feature 23: WVRSHBW, Coefficient: -0.0919
Feature 24: FB

## RFE approach

In [19]:
from sklearn.feature_selection import RFE

In [ ]:
# Lists to store results
num_features = []
performances = []

for i in range(1, X_train.shape[1] + 1):
    rfe = RFE(model, i)
    fit = rfe.fit(X_train, y_train)
    
    pred = fit.predict(X_test[:,fit.support_])
    f1 = f1_score(test_target, test_predictions)
    num_features.append(i)
    performances.append(f1)RidgeClassifier